In [19]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [25]:
"""
요약
ConversationSummaryBufferMemory 는
ConversationSummaryMemory, ConversationBufferMemory 의 결합

limit에 다다르면 오래된 메시지들은 요약한다.
최근 메세지는 보여주고, 오래된 메세지는 요약함.
"""
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

In [37]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True
)
# max_token_limit 은 token Limit 임

In [38]:
def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

In [39]:
add_message("Hi I'm dooseop, I live in Seoul", "Wow that is so cool!!")

In [40]:
add_message("Seoul is so pretty", "I with I could go!!")

In [41]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as dooseop and mentions they live in Seoul.'),
  AIMessage(content='Wow that is so cool!!'),
  HumanMessage(content='Seoul is so pretty'),
  AIMessage(content='I with I could go!!')]}